<a href="https://colab.research.google.com/github/jdscifi/AutoCaptcha/blob/developement/StockIt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
% pip install yfinance
% pip install pandas
% pip install --upgrade pandas-datareader

     |████████████████████████████████| 6.3 MB 7.6 MB/s 
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
     |████████████████████████████████| 109 kB 8.1 MB/s 
  Attempting uninstall: pandas-datareader
    Found existing installation: pandas-datareader 0.9.0
    Uninstalling pandas-datareader-0.9.0:
      Successfully uninstalled pandas-datareader-0.9.0


# Data handling and importing all the required library

In [ ]:
import pandas as pd
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
import os
import time
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup
import datetime
import logging
import threading as thr
import yfinance as yf
from IPython.display import clear_output 
from IPython.core.display import display, HTML


Now we are setting paparmeter for displaying News, where we want 7 days news which will not contain today's date. Also, logging/saving article in log file and using method which will check whether all elements are True, potentially over an axis.

In [ ]:
NEWSDAYSLIMIT = 7
lock = thr.Lock()
date_threshold = datetime.datetime.now() - datetime.timedelta(NEWSDAYSLIMIT)
logging.basicConfig(filename="StockIt.log")
all_articles = pd.DataFrame()
stop_flag = False

# Functions to scrape the articles from the PR News Site

In [ ]:
def store_data():
    try:
        global all_articles
        all_articles['time'] = all_articles["time"].dt.strftime("%b %d %Y %H:%M:%S")
        all_articles.to_csv("articles.csv", index=False)
    except Exception:
        logging.exception("Exception store_data\n", exc_info=True)


def parse_page(page_html,organisation):
    try:
        global all_articles, stop_flag, lock, date_threshold
        soup_parser = BeautifulSoup(page_html, features="html.parser")
        news_articles = soup_parser.findAll('a', class_='newsreleaseconsolidatelink')
        page_articles = []
        for article in news_articles:
            try:
                article_data = {"link": article['href'], "header": "", "time": None, "thumbnail": "", "summary": "","org":organisation}
                try:
                    date_tag = article.select("h3 > small")[0].text
                    article_data["time"] = datetime.datetime.strptime(date_tag, "%b %d, %Y, %H:%M ET")
                except Exception:
                    today = datetime.datetime.today()
                    article_data["time"] = datetime.datetime.strptime(date_tag, "%H:%M ET")
                    article_data["time"] = article_data["time"].replace(year=today.year, month=today.month,
                                                                        day=today.day)
                if article_data["time"] < date_threshold:
                    stop_flag = True
                    break
                articles_images = article.findAll("img")
                if len(articles_images) > 0:
                    article_data["thumbnail"] = article.findAll("img")[0]['src']
                article_data["header"] = str(article.select("h3")[0].text).split("\n")[2]
                article.small.decompose()
                article_data["summary"] = article.select("p")[0].text
                page_articles.append(article_data)
            except Exception:
                logging.exception("Exception article read!\n", exc_info=True)
        with lock:
            all_articles = all_articles.append(pd.DataFrame(page_articles))
            last_read = all_articles['time'].min()
            clear_output()
            print("Earliest News: {}\nNumber of articles scrapped: {}".format(last_read, len(all_articles)))
    except Exception:
        logging.exception("Exception parse_page", exc_info=True)


def collect_articles(page_no_,url_,organisation=None):
    try:
        attempts = 0
        while attempts < 5:
            try:
                temp = url_
                if page_no_ > 0:
                    temp = url_.format(page_no_)
                logging.info(temp)
                result = rq.get(temp)
                if result.status_code == 200:
                    parse_page(result.text,organisation)
                    break
                else:
                    logging.info("For page number {}, response received: {}\n".format(page_no_, str(result)))
            except Exception:
                logging.exception("Exception collect_articles\n", exc_info=True)
                attempts += 1
        if attempts > 5:
            logging.error("Could not read page number: {}".format(page_no_))
    except Exception:
        logging.exception("Exception collect_articles", exc_info=True)

# A threaded call 

In [ ]:
try:
    page_no = 1
    while not stop_flag:
        thread = thr.Thread(target=collect_articles, args=(page_no,"https://www.prnewswire.com/news-releases/news-releases-list/?page={}&pagesize=100"))
        while thr.active_count() > 20:
            time.sleep(10)
        page_no += 1
        thread.start()
    wep = 0
    while thr.active_count() != 1:
        wep += 1
        if wep >= 10:
            break
        time.sleep(10)
    store_data()
except KeyboardInterrupt:
    store_data()
except Exception as e:
    print(e)
    logging.exception("main exception!", exc_info=True)

Earliest News: 2021-11-22 23:40:00
Number of articles scrapped: 3142


In [ ]:
try:
    article_start = 0
    while True:
        clear_output()
        articles_html = """"""
        for index, na in all_articles.iloc[article_start: article_start+5].iterrows():
            art = """<a style="text-decoration:none;" href="{}"
        <h4><img width="50px" src="{}"> {}</h4>
        <p>{}</p>
        </a>
        <hr>
        <br>
        """.format(na['link'],na['thumbnail'],na['header'],na['summary'])
    #print(art)
            articles_html = articles_html + art
# print(articles_html)
        display(HTML(articles_html))
        user_input= input("Read more? y/n")
        if user_input != "y":
            break
        article_start +=5
        if (article_start + 5) > len(all_articles):
            break
except KeyboardInterrupt:
    print("Done Viewing")
except Exception as e:
    print(e)
    logging.exception("Exception News view", exc_info=True)

Read more? y/nn


In [ ]:
date_threshold = datetime.datetime.today() - datetime.timedelta(days=90)
pepsi_articles = []
print(len(all_articles))
try:
    pepsi_news = rq.get("https://www.prnewswire.com/news/pepsico%2C-inc./?page=1&pagesize=100")
    if pepsi_news.status_code == 200:
        parse_page(pepsi_news.text,"Pepsi")
    print(len(all_articles))
    coca_news = rq.get("https://www.prnewswire.com/search/news/?keyword=Coca%20Cola")
    if pepsi_news.status_code == 200:
        parse_page(pepsi_news.text,"Coca cola")
    print(len(all_articles))
except Exception as e:
    print(e)
    logging.exception("pepsi collection", exc_info=True)
finally:
    store_data()
    #

3142
3154
3166


In [ ]:
try:
    article_start = 0
    pc_news_articles = all_articles[all_articles['org']!=None]
    while True:
        clear_output()
        pc_articles_html = """"""
        for index, na in pc_news_articles.iloc[article_start:article_start+5].iterrows():
            art = """<a style="text-decoration:none;" href="{}"
        <h4><img width="50px" src="{}"> {}</h4>
        <p>{}</p>
        </a>
        <hr>
        <br>
        """.format(na['link'],na['thumbnail'],na['header'],na['summary'])
    #print(art)
            pc_articles_html = articles_html + art
# print(articles_html)
        display(HTML(pc_articles_html))
        user_input= input("Read more? y/n")
        if user_input != "y":
            break
        article_start +=5
        if (article_start + 5) > len(all_articles):
            break
except KeyboardInterrupt:
    print("Done Viewing")
except Exception as e:
    print(e)
    logging.exception("Exception News view", exc_info=True)

Read more? y/nn


Now, we mention Pepsi symbol as PEP and further retrieve 6 months data, in which we display top 10 data.

In [ ]:
pepsi = yf.Ticker("PEP")
pepsi_hist = pepsi.history(period="6mo")
pepsi_hist.head(10)
pepsi_hist.to_csv("pepsi.csv")

Same we will do for Cocacola, by which we can see Cocacola top 10 data. We can see, data consists of, Date, opening price of stock, closing price and High and low price(maximum and minimum), total voulme , dividends and splits of Stock.volume

In [ ]:
coca = yf.Ticker("KO")
coca_hist = coca.history(period="6mo")
coca_hist.head(10)
coca_hist.to_csv("cocacola.csv")

Below line chart of Pepsi Vs Cocacola Stock price illustrates, the stock price from June 2021 to November 2021. In which, we can see Pepsi has been the highest stock price as compare to Cocacola,whereas we can see Pepsi stock has increased continously and in opposite to that Cocacola has fluctuating and stoped as price where it has seen in June.*italicized text*

Below line graph of Pepsi Vs Cocacola Market Cap depicts, the price distribution from June 2021 to November 2021. In which, we can see Cocacola has been the highest price as compare to Pepsi,whereas after November in the end both has almost same price. We have seen more variation in the Cocacola price as compare to Pepsi.

In [ ]:
stp = figure(plot_width = 600, plot_height = 600, title = 'Pepsi vs Cocacola Stock Price', x_axis_label = 'Date',x_axis_type="datetime",y_axis_type="log", y_axis_label = 'Price')
stp.line(pepsi_hist.index, pepsi_hist["Close"], line_color="red", legend_label="Pepsi")
stp.line(coca_hist.index, coca_hist["Close"], line_color="blue", legend_label="Cocacola")
p = figure(plot_width = 800, plot_height = 600, title = 'Pepsi vs Cocacola Market Cap', x_axis_label = 'Date',x_axis_type="datetime",y_axis_type="log", y_axis_label = 'Market Cap')
p.line(pepsi_hist.index, pepsi_hist["Close"] * pepsi_hist["Volume"] ,line_color="red", legend_label="Pepsi")
p.line(coca_hist.index, coca_hist["Close"] * coca_hist["Volume"] ,line_color="blue", legend_label="Cocacola")
output_notebook()
show(row(stp, p))

Below figure of Pepsi Vs Cocacola Market Volume defines, the Volume distribution from June 2021 to November 2021. In which, we can see Cocacola has been the highest in terms of volume as compare to Pepsi. We have seen more variation in Pepsi Volume as compare to Cocacola. Volume is seen fluctuating always.

In [ ]:
p = figure(plot_width = 1000, plot_height = 600, title = 'Pepsi vs Cocacola Market Volume', x_axis_label = 'Date',x_axis_type="datetime",y_axis_type="log", y_axis_label = 'Volume')
p.line(pepsi_hist.index, pepsi_hist["Volume"] ,line_color="red", legend_label="Pepsi")
p.line(coca_hist.index, coca_hist["Volume"] ,line_color="blue", legend_label="Coca Cola")
output_notebook()
show(p)